In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
import time
import pickle
import datetime

import torch

plt.rc('font', size = 9)
plt.rc('font', family='sans serif')
plt.rcParams['pdf.fonttype']=42
plt.rcParams['ps.fonttype']=42
plt.rcParams['legend.frameon']=False
plt.rcParams['axes.grid']=False
plt.rcParams['legend.markerscale']=0.5
plt.rcParams['savefig.dpi']=600
sns.set_style("ticks")

In [2]:
X = torch.rand(32, 10080, 2)

In [52]:
import sys
sys.path.append('/home/ngrav/project/')
from wearables.scripts import utils as wearutils
from wearables.scripts import data_v42 as weardata
from wearables.scripts import train_v42 as weartrain
from wearables.scripts import eval_v42 as weareval
from wearables.scripts import model as wearmodels
from wearables.scripts import DTW as weardtw

from wearables.scripts import data_augmentation as aug

In [50]:
# load data

model_fp = '/home/ngrav/project/wearables/model_zoo'

bst_trainer = os.path.join(model_fp, 'trainer_cnnv43l1l2_GA5.pkl')
bst_modelpkl = os.path.join(model_fp, '6143-cnnv43l1l2_GA5.pkl')


In [53]:
# # fix potential issue with net
net = wearmodels.CNN(2, 10080, 1, [(1, 32), (1, 64), (1, 128), (3, 256)])

# best CNN
_, md_model, trainer = weardtw.data_from_trainer(bst_trainer)

# fix issue with net
trainer.model = net

# call eval to get output, modify for return_hidden = True option
evaluation = weareval.eval_trained(trainer, split='train',
                                   modelpkl=bst_modelpkl,
                                   two_outputs=True)

[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
/home/ngrav/miniconda3/envs/wearables/lib/python3.8/site-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)
/home/ngrav/miniconda3/envs/wearables/lib/python3.8/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


# dev augmentations 

In [70]:
import importlib
importlib.reload(aug)

<module 'wearables.scripts.data_augmentation' from '/home/ngrav/project/wearables/scripts/data_augmentation.py'>

In [75]:
def printdata(X):
    print('X:\n', X, '\n\nShape:\n', X.shape)

In [76]:
batch = next(iter(trainer.data.train_dl))
X = batch['x']
printdata(X)

[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


X:
 tensor([[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0000, 3.3322, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[3.6889, 3.3322, 0.0000,  ..., 3.7612, 2.5649, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        ...,

        [[3.6636, 4.9200, 5.0999,  ..., 5.1705, 5.0434, 3.9120],
         [2.8959, 2.8530, 2.8082,  ..., 2.3145, 2.3514, 1.6448]],

        [[3.2189, 0.0000, 4.5850,  ..., 5.9789, 5.0039, 3.2189],
         [2.6868, 2.7139, 2.6010,  ..., 2.4738, 2.0596, 2.1096]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]]) 

Shape:
 torch.Size([64, 2, 10080])


In [77]:
printdata(aug.jitter(X))
printdata(aug.scaling(X))

X:
 tensor([[[-1.5594e-02, -1.7587e-02,  8.6085e-02,  ..., -2.0962e-02,
          -1.9111e-02, -7.6582e-04],
         [ 6.1087e-02,  1.0482e-02, -3.7301e-02,  ..., -7.7162e-03,
           1.3235e-04,  5.4338e-03]],

        [[-7.0354e-03,  3.3235e+00, -1.3301e-02,  ..., -1.3961e-02,
          -1.6914e-02, -1.0174e-02],
         [ 2.4679e-03,  5.3518e-03, -1.9156e-02,  ..., -4.7091e-02,
          -2.6144e-03,  4.3014e-02]],

        [[ 3.6559e+00,  3.3348e+00, -4.1193e-02,  ...,  3.7323e+00,
           2.5115e+00, -2.5950e-02],
         [-2.4043e-02,  1.6654e-02,  9.2061e-03,  ..., -2.0608e-02,
           1.6129e-02,  2.9876e-02]],

        ...,

        [[ 3.7017e+00,  4.9346e+00,  5.1473e+00,  ...,  5.1712e+00,
           5.0420e+00,  3.8430e+00],
         [ 2.9635e+00,  2.8244e+00,  2.7930e+00,  ...,  2.2904e+00,
           2.4218e+00,  1.6462e+00]],

        [[ 3.2455e+00, -2.1872e-02,  4.5867e+00,  ...,  5.9402e+00,
           5.0312e+00,  3.2091e+00],
         [ 2.6971e+00,  2.674

In [104]:
x.shape

torch.Size([64, 2, 10080])

In [103]:
for i, pat in enumerate(X):
    print(i, pat.shape)

0 torch.Size([2, 10080])
1 torch.Size([2, 10080])
2 torch.Size([2, 10080])
3 torch.Size([2, 10080])
4 torch.Size([2, 10080])
5 torch.Size([2, 10080])
6 torch.Size([2, 10080])
7 torch.Size([2, 10080])
8 torch.Size([2, 10080])
9 torch.Size([2, 10080])
10 torch.Size([2, 10080])
11 torch.Size([2, 10080])
12 torch.Size([2, 10080])
13 torch.Size([2, 10080])
14 torch.Size([2, 10080])
15 torch.Size([2, 10080])
16 torch.Size([2, 10080])
17 torch.Size([2, 10080])
18 torch.Size([2, 10080])
19 torch.Size([2, 10080])
20 torch.Size([2, 10080])
21 torch.Size([2, 10080])
22 torch.Size([2, 10080])
23 torch.Size([2, 10080])
24 torch.Size([2, 10080])
25 torch.Size([2, 10080])
26 torch.Size([2, 10080])
27 torch.Size([2, 10080])
28 torch.Size([2, 10080])
29 torch.Size([2, 10080])
30 torch.Size([2, 10080])
31 torch.Size([2, 10080])
32 torch.Size([2, 10080])
33 torch.Size([2, 10080])
34 torch.Size([2, 10080])
35 torch.Size([2, 10080])
36 torch.Size([2, 10080])
37 torch.Size([2, 10080])
38 torch.Size([2, 1008

In [94]:
(strt + 9072).dtype

torch.int64

In [85]:
torch.randint(10080-9072, size=(32,))

tensor([378, 163, 149,  39, 152,  55, 511,  69,   1, 975, 348, 733, 402, 301,
         43, 676, 320, 541,  98,  60, 731, 106, 272, 344, 182, 316, 345, 474,
        867, 648, 257, 869])

In [57]:
Xprime_sc = aug.scaling(X)
Xprime_sc[0, 0, :10]

tensor([[1.1112, 1.1112, 1.1112,  ..., 1.1112, 1.1112, 1.1112]])


In [45]:
Xprime_sc[0, :, :]

tensor([[0.0697, 0.0431],
        [0.7462, 0.2365],
        [1.0573, 0.3995],
        ...,
        [0.4024, 0.3984],
        [0.2614, 0.9320],
        [0.8369, 0.5396]])

In [60]:
a = torch.rand(32, 1).repeat(1, 2)

In [69]:
(a.unsqueeze(-1)*X[:32, :, :]).shape

torch.Size([32, 2, 10080])

In [43]:
factor = torch.normal(1.0, sigma, size=(x.shape[0], 1))
factor = factor.repeat(1, x.shape[2]).unsqueeze(1)
return factor * x

tensor([[[0.0596, 0.0369],
         [0.6377, 0.2021],
         [0.9036, 0.3414],
         ...,
         [0.3439, 0.3405],
         [0.2234, 0.7966],
         [0.7153, 0.4612]],

        [[0.2874, 0.6301],
         [0.3652, 0.7726],
         [0.9882, 0.7088],
         ...,
         [0.6019, 0.1745],
         [0.5133, 0.1650],
         [0.0945, 0.1263]],

        [[0.9639, 0.8774],
         [0.0775, 0.2425],
         [0.0945, 0.7686],
         ...,
         [0.4086, 0.9040],
         [0.5888, 0.8940],
         [0.8535, 0.4663]],

        ...,

        [[0.5013, 0.8265],
         [0.1360, 0.5555],
         [0.9088, 0.3801],
         ...,
         [0.1671, 0.0270],
         [0.7479, 0.5993],
         [0.9830, 0.1802]],

        [[0.9886, 0.5755],
         [0.6431, 0.1376],
         [0.5719, 0.0389],
         ...,
         [0.3919, 0.3656],
         [0.5025, 0.9074],
         [0.6556, 0.6916]],

        [[0.5810, 0.7003],
         [0.5114, 0.9149],
         [0.5998, 0.7675],
         ...,
 